In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [5]:
anon = pd.read_csv('../Data_Anom/anonym212_385', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [6]:
print(anon)

             id                date  latitude  longitude  week
0         34779 2015-04-09 07:07:00  4.956061  45.680407    15
2         17062 2015-03-07 07:03:00  4.841085  45.771106    10
3         68773 2015-03-07 20:59:00 -0.644400  45.011832    10
4         11683 2015-04-11 17:05:00  4.840916  45.770817    15
5          8895 2015-03-25 15:35:00  4.926721  45.780244    13
...         ...                 ...       ...        ...   ...
34551843  46044 2015-05-06 14:26:00  4.867849  45.750828    19
34551845  61133 2015-05-02 13:40:00  4.756775  45.995724    18
34551846  43156 2015-04-18 19:52:00  4.834565  45.783475    16
34551847  95716 2015-04-30 22:35:00  4.853006  45.738505    18
34551848  49859 2015-04-22 08:09:00  4.867437  45.751008    17

[27641479 rows x 5 columns]


In [7]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [9]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['10263' '10395' '10657' '1071' '1072' '11061' '11101' '1131' '11360'
 '11363' '11463' '11528' '11603' '11668' '11683' '11848' '11861' '11907'
 '12008' '1204' '12167' '12202' '12494' '1260' '12983' '13139' '13374'
 '13645' '13833' '13864' '13876' '14164' '1430' '14597' '14876' '15020'
 '15184' '15276' '15290' '15589' '15685' '1573' '15829' '15900' '15914'
 '16129' '16564' '16594' '16825' '17062' '17322' '17521' '179' '17985'
 '18138' '18208' '19043' '19201' '19207' '19347' '19432' '19588' '19743'
 '19954' '20077' '20180' '20248' '20806' '20877' '2100' '21495' '21561'
 '21774' '21930' '22068' '22151' '22176' '22333' '22759' '22876' '23075'
 '23444' '24163' '24202' '24551' '24860' '24916' '25077' '25417' '256

In [11]:
print(id_week_anon)

id
10263    [18]
10395    [14]
10657    [12]
1071     [15]
1072     [17]
         ... 
98335    [17]
98984    [19]
99235    [12]
99586    [14]
99680    [12]
Name: week, Length: 449, dtype: object


In [12]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [14]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            # Calculate distances for each row in the anonymized data
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)) ,1/2)
            
            list_distance.append(distance*delta_ratio*delta_hour*delta_time) # Combine distance measures and add to the list
            
        index = list_distance.index(min(list_distance)) # Identify the index of the minimum distance in the list (list_distance).
        l = id_week_anon[group_week.id[index]] # List of weeks associated with the anonymized 'id' at the identified index.
        # Finding the Closest Match with Original Week:
        while ((w not in l)):
            # Update the distance for the current index
            list_distance[index] = 1000000000
            # Find the new minimum distance and corresponding list of weeks
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
        # If the original week is found in the updated list (l), update the 'new_id' in the original dataset (ori_mean) 
        # with the anonymized 'id' at the identified index.
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

449
{'1': {'2015-10': ['82781'], '2015-11': ['95233'], '2015-12': ['36211'], '2015-13': ['26448'], '2015-14': ['6412'], '2015-15': ['80046'], '2015-16': ['11907'], '2015-17': ['24163'], '2015-18': ['15184'], '2015-19': ['17985'], '2015-20': ['65467']}, '2': {'2015-10': ['28169'], '2015-11': ['83873'], '2015-12': ['63717'], '2015-13': ['46326'], '2015-14': ['8574'], '2015-15': ['12983'], '2015-16': ['34667'], '2015-17': ['67638'], '2015-18': ['27712'], '2015-19': ['61522'], '2015-20': ['65815']}, '107': {'2015-10': ['38391'], '2015-11': ['15020'], '2015-12': ['16564']}, '98': {'2015-10': ['47667'], '2015-11': ['5893']}, '21': {'2015-10': ['34258'], '2015-11': ['48376']}, '5': {'2015-10': ['19743'], '2015-11': ['49400']}, '9': {'2015-10': ['17062'], '2015-11': ['85837'], '2015-12': ['64582'], '2015-13': ['88687'], '2015-14': ['72402'], '2015-15': ['11683'], '2015-16': ['95039'], '2015-19': ['77399']}, '6': {'2015-10': ['54004'], '2015-11': ['15685'], '2015-12': ['4261'], '2015-13': ['441

In [8]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "82781"
        ],
        "2015-11": [
            "95233"
        ],
        "2015-12": [
            "36211"
        ],
        "2015-13": [
            "26448"
        ],
        "2015-14": [
            "6412"
        ],
        "2015-15": [
            "80046"
        ],
        "2015-16": [
            "11907"
        ],
        "2015-17": [
            "24163"
        ],
        "2015-18": [
            "15184"
        ],
        "2015-19": [
            "17985"
        ],
        "2015-20": [
            "65467"
        ]
    },
    "2": {
        "2015-10": [
            "28169"
        ],
        "2015-11": [
            "83873"
        ],
        "2015-12": [
            "63717"
        ],
        "2015-13": [
            "46326"
        ],
        "2015-14": [
            "8574"
        ],
        "2015-15": [
            "12983"
        ],
        "2015-16": [
            "34667"
        ],
        "2015-17": [
            "67